# Logistic Regression example

In [11]:
// cargo install the newest version
// this may take a while, needs to download and install the library
:dep smartcore = { version = "0.3", features = ["datasets"] }

In [12]:
use smartcore::dataset::iris::load_dataset as iris_load;
use smartcore::dataset::breast_cancer::load_dataset as bc_load;
use smartcore::dataset::diabetes::load_dataset as db_load;
use smartcore::linear::logistic_regression::LogisticRegression;
use smartcore::linalg::basic::matrix::DenseMatrix;
// Model performance
use smartcore::metrics::accuracy;

use smartcore::model_selection::{cross_val_predict, cross_validate, KFold};

## basic prediction and accuracy

In [13]:
// Load Iris dataset
let iris_dataset = iris_load();

// Turn Iris dataset into NxM matrix
// Input data
let x: DenseMatrix<f32> = DenseMatrix::new(
    iris_dataset.num_samples,      // num rows
    iris_dataset.num_features,     // num columns
    iris_dataset.data,             // data as Vec
    false,                         // column_major
);
// These are our target class labels
let y: Vec<u32> = iris_dataset.target;

// Fit Logistic Regression to Iris dataset
let lr = LogisticRegression::fit(&x, &y, Default::default()).unwrap();
let y_hat = lr.predict(&x).unwrap(); // Predict class labels

// Calculate training error
println!("accuracy: {}", accuracy(&y, &y_hat)); // Prints 0.98

accuracy: 0.9866666666666667


## basic cross validation

In [14]:
// we need to import this as LogisticRegression is a SupervisedEstimator
// and we need its traits
use smartcore::api::SupervisedEstimator

// This example is expired by
// https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_predict.html
// Load dataset
let breast_cancer_data = bc_load();
let x: DenseMatrix<f32> = DenseMatrix::new(
    breast_cancer_data.num_samples,
    breast_cancer_data.num_features,
    breast_cancer_data.data,
    false
);
// These are our target values
let y = breast_cancer_data.target;
// cross-validated estimator
let results = cross_validate(
    LogisticRegression::new(),
    &x,
    &y,
    Default::default(),
    &KFold::default().with_n_splits(3),
    &accuracy,
)
.unwrap();
println!(
    "Test score: {}, training score: {}",
    results.mean_test_score(),
    results.mean_train_score()
);


Test score: 0.9613478139793928, training score: 0.9622182104337361


## basic cross validation with prediction

In [15]:
use smartcore::linear::linear_regression::LinearRegression;

// Load Diabetes dataset
let diabetes_data = db_load();
let x = DenseMatrix::new(
    diabetes_data.num_samples,
    diabetes_data.num_features,
    diabetes_data.data,
    false
);

// These are our target values
let y = diabetes_data.target;

// Generate cross-validated estimates for each input data point
let y_hat = cross_val_predict(
    LinearRegression::new(),
    &x,
    &y,
    Default::default(),
    &KFold::default().with_n_splits(10),
)
.unwrap();


println!(
    "y hat {:?}",
    &y_hat
);


y hat [202, 71, 178, 166, 128, 108, 71, 121, 156, 217, 96, 100, 115, 162, 105, 175, 211, 184, 149, 124, 120, 83, 116, 252, 166, 146, 92, 185, 127, 179, 161, 71, 253, 114, 77, 82, 208, 155, 240, 137, 150, 71, 146, 75, 220, 126, 142, 108, 73, 190, 156, 166, 135, 162, 134, 67, 209, 84, 107, 136, 116, 182, 62, 96, 115, 189, 150, 123, 117, 119, 78, 231, 142, 125, 149, 128, 195, 74, 169, 91, 175, 123, 63, 149, 54, 168, 40, 150, 84, 105, 81, 186, 197, 58, 109, 119, 206, 208, 121, 139, 166, 108, 146, 158, 152, 117, 70, 156, 231, 144, 34, 123, 155, 207, 288, 183, 214, 235, 167, 151, 157, 202, 221, 185, 170, 183, 57, 111, 90, 209, 247, 69, 114, 68, 141, 241, 57, 233, 246, 253, 154, 219, 169, 117, 180, 243, 191, 234, 112, 184, 211, 147, 191, 120, 152, 198, 148, 122, 84, 239, 87, 234, 146, 195, 146, 80, 57, 261, 222, 220, 47, 84, 222, 91, 166, 118, 156, 225, 97, 165, 179, 88, 173, 156, 198, 191, 200, 66, 156, 115, 196, 125, 89, 142, 150, 170, 102, 189, 143, 182, 89, 69, 160, 195, 180, 232, 161, 21

In [10]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }

use plotters::prelude::*;

//Assemble XY dataset for the scatter plot
let xy: Vec<Vec<u32>> = y.clone()
        .into_iter()
        .zip(y_hat.clone().into_iter())
        .map(|(x1, x2)| vec![x1, x2])
        .collect();

let diff: Vec<Vec<u32>> = y.clone()
        .into_iter()
        .zip(y_hat.clone().into_iter())
        .map(|(x1, x2)| vec![(u32::max(x1, x2) - u32::min(x1, x2)), x2])
        .collect();

// println!("{:?}",
//     &xy
// );

evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Y and Y_hat (green) and their difference (blue)", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0u32..350u32, 0u32..400u32)?;
    
    chart.configure_mesh()
        .x_desc("Y")
        .y_desc("Y_hat")
        .draw()?;
    
    chart.draw_series(xy.iter().map(|x| Circle::new((x[0], x[1]), 3, GREEN.filled())))?;
    
    chart.draw_series(diff.iter().map(|x| Circle::new((x[0], x[1]), 3, BLUE.filled())))?;
    
    // You can alawys freely draw on the drawing backend
    let area = chart.plotting_area();
    let sd = 0.13;
    let two_sigma = sd * 2.0;
    area.draw(&Rectangle::new(
        [((0.5 - two_sigma) as u32, (0.5 - two_sigma)  as u32),
            ((0.5 + two_sigma)  as u32, (0.5 + two_sigma)  as u32)], 
        RED.mix(0.3).filled())
    )?;
    
    Ok(())
}).style("width: 60%")

Y and Y_hat (green) and their difference (blue)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Y_hat
 
 
Y
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
350
 
 
 
400
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
350